In [12]:
from langchain_ollama import OllamaLLM
from langchain_core.runnables import RunnableSequence, RunnableBranch
from langchain_core.prompts import ChatPromptTemplate
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

In [18]:
llm = OllamaLLM(model="llama3:8b")

In [21]:
identification_template = ChatPromptTemplate([
    ("system", "you are an chat classifier to see if the user asks question about sql or general query return only True/False"),
    ("human", "please classify if my question: {question} is about asking sql query or general")
])
identification_chain = RunnableSequence(identification_template, llm)

In [23]:
general_template = ChatPromptTemplate([
    ("system", "you are an helpful AI agent answers user questions."),
    ("human", "please classify doubt: {question}")
])
general_chain = RunnableSequence(general_template, llm)

In [ ]:
RunnableBranch(
    (lambda x: isinstance(x, str), lambda x: x.upper()),
    (lambda x: isinstance(x, int), lambda x: x + 1),
    (lambda x: isinstance(x, float), lambda x: x * 2),
    lambda x: "goodbye",
)

In [19]:
def chat(state: MessagesState):
    response = llm.invoke(state["messages"])
    return {"messages": response}

In [ ]:
workflow = StateGraph(state_schema=MessagesState)
workflow.add_node("llm", chat)
workflow.add_edge(START, "llm")

memory = MemorySaver()

app = workflow.compile(checkpointer=memory)